# Intrusion Detection System (IDS)
Βασισμένο στο dataset [pcap_data.csv](https://github.com/kdemertzis/EKPA/blob/main/Data/pcap_data.csv)
και το πρότυπο κώδικα του αρχείου `IDS_example.py`. Τρέχει στο Google Colab.

In [ ]:
# Εγκατάσταση και imports
!pip install tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import class_weight
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# Φόρτωση dataset από GitHub
url = 'https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/pcap_data.csv'
df = pd.read_csv(url)
df = df.dropna()
df = df[df['Label'] != 'DrDoS_LDAP']
df = df.sample(frac=1, random_state=42)
labels = df['Label'].copy()

In [ ]:
# Χαρτογράφηση ετικετών σε αριθμούς
label_map = {'WebDDoS':0, 'BENIGN':1, 'UDP-lag':2, 'DrDoS_NTP':3, 'Syn':4,
             'DrDoS_SSDP':5, 'DrDoS_UDP':6, 'DrDoS_NetBIOS':7, 'DrDoS_MSSQL':8,
             'DrDoS_SNMP':9, 'TFTP':10, 'DrDoS_DNS':11}
df['Label'] = df['Label'].map(label_map)
y = df['Label'].values
X = df.drop(columns=['Label']).values
# Κανονικοποίηση
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

In [ ]:
# Split δεδομένων
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)
y_train_cat = to_categorical(y_train, num_classes=12)
y_val_cat = to_categorical(y_val, num_classes=12)
y_test_cat = to_categorical(y_test, num_classes=12)

In [ ]:
# Μοντέλο Conv1D
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(X.shape[1], 1)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(12, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Εκπαίδευση
history = model.fit(X_train[..., np.newaxis], y_train_cat, epochs=30, batch_size=256,
                    validation_data=(X_val[..., np.newaxis], y_val_cat),
                    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)])

In [ ]:
# Confusion Matrix
pred = model.predict(X_test[..., np.newaxis])
y_pred = pred.argmax(axis=1)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_map.keys())
disp.plot(xticks_rotation=45, cmap='Blues')
plt.show()